# Парсинг результатов президенстких выборов

## Импорты

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

import multiprocessing
from itertools import starmap
from tqdm import tqdm

## Вспомогательные функции и константы

In [2]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.214 Safari/537.36"
}

In [3]:
start_url = "http://notelections.online/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227"

## Парсинг

### Получение ссылок на страницы с таблицами избирательных комиссий

На начальной странице (```start_url```) есть список с ссылками на страницы с таблицами по отдельно взятым регионам. На каждом такой странице только для данного региона доступен список страниц с таблицами для отдельного взятого района.

In [4]:
start_r = requests.get(start_url, headers=HEADERS)

In [5]:
soup = BeautifulSoup(start_r.content, 'lxml')

Получаем список ссылок на информацию по регионам 

In [6]:
states_links = [(link.text, "http://notelections.online" + link.get("href")) for link in soup.find_all("a", style="text-decoration: none")]

Парсим таблицы для каждого УИК в каждом ТИК всех регионов и соединяем в один ```pd.DataFrame```

In [7]:
def extract_votes_for_candidate(field: str):
    
    return int(field.split()[0])

def get_df_for_hood(name, link):
    df = pd.read_html(link, encoding='cp1251', match="Сумма")[0]
    
    df.index = df.iloc[:, 1]
    df.index.name = None
    # Remove unnecessary columns
    df = df.iloc[:, 3:]
    # For each candidate have field in format "<votes> <votes, %>"
    # Have to get only int(<votes>)

    df.columns = pd.MultiIndex.from_product(
        [[name], df.columns],
        names=["ТИК", "УИК"]
    )
    
    return df

def get_table_for_state(name, link):
    state_r = requests.get(link, headers=HEADERS)
    state_soup = BeautifulSoup(state_r.content, 'lxml')
    
    hood_links = [(link.text, "http://notelections.online" + link.get("href")) for link in state_soup.find_all("a", style="text-decoration: none")]
    if hood_links:
        hood_dfs = starmap(get_df_for_hood, hood_links)
        state_df = pd.concat(hood_dfs, axis=1, join="outer")
    else:
        state_df = get_df_for_hood(None, link)
        
    return pd.concat([state_df], axis=1, keys=[name], names=["Регион"])

In [8]:
pool = multiprocessing.Pool()
state_dfs = pool.starmap(get_table_for_state, states_links)
df = pd.concat(state_dfs, axis=1, join="outer")

In [9]:
df = df.transpose()

In [10]:
df.head()

Число избирателей, включенных в список избирателей  \
Регион                     ТИК        УИК                                                         
Республика Адыгея (Адыгея) Адыгейская УИК №1                                               2256   
                                      УИК №2                                               2700   
                                      УИК №3                                               2858   
                                      УИК №4                                               2066   
                                      УИК №5                                                700   

                                             Число избирательных бюллетеней, полученных участковой избирательной комиссией  \
Регион                     ТИК        УИК                                                                                    
Республика Адыгея (Адыгея) Адыгейская УИК №1                                               2181                              
                                      УИК №2                                               2633                              
                                      УИК №3                                               2752                              
                                      УИК №4                                               2034                              
                                      УИК №5                                                714                              

                                             Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно  \
Регион                     ТИК        УИК                                                                                     
Республика Адыгея (Адыгея) Адыгейская УИК №1                                                  0                               
                                      УИК №2                                                  0                               
                                      УИК №3                                                  0                               
                                      УИК №4                                                  0                               
                                      УИК №5                                                  0                               

                                             Число избирательных бюллетеней, выданных в помещении для голосования в день голосования  \
Регион                     ТИК        УИК                                                                                              
Республика Адыгея (Адыгея) Адыгейская УИК №1                                               2107                                        
                                      УИК №2                                               2575                                        
                                      УИК №3                                               2664                                        
                                      УИК №4                                               1857                                        
                                      УИК №5                                                676                                        

                                             Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования  \
Регион                     ТИК        УИК                                                                                                
Республика Адыгея (Адыгея) Адыгейская УИК №1                                                 62                                          
                                      УИК №2                                                 41                                          
                                      УИК №3                             

In [11]:
df.iloc[:, 12:] = df.iloc[:, 12:].applymap(extract_votes_for_candidate)
df.head()

Число избирателей, включенных в список избирателей  \
Регион                     ТИК        УИК                                                         
Республика Адыгея (Адыгея) Адыгейская УИК №1                                               2256   
                                      УИК №2                                               2700   
                                      УИК №3                                               2858   
                                      УИК №4                                               2066   
                                      УИК №5                                                700   

                                             Число избирательных бюллетеней, полученных участковой избирательной комиссией  \
Регион                     ТИК        УИК                                                                                    
Республика Адыгея (Адыгея) Адыгейская УИК №1                                               2181                              
                                      УИК №2                                               2633                              
                                      УИК №3                                               2752                              
                                      УИК №4                                               2034                              
                                      УИК №5                                                714                              

                                             Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно  \
Регион                     ТИК        УИК                                                                                     
Республика Адыгея (Адыгея) Адыгейская УИК №1                                                  0                               
                                      УИК №2                                                  0                               
                                      УИК №3                                                  0                               
                                      УИК №4                                                  0                               
                                      УИК №5                                                  0                               

                                             Число избирательных бюллетеней, выданных в помещении для голосования в день голосования  \
Регион                     ТИК        УИК                                                                                              
Республика Адыгея (Адыгея) Адыгейская УИК №1                                               2107                                        
                                      УИК №2                                               2575                                        
                                      УИК №3                                               2664                                        
                                      УИК №4                                               1857                                        
                                      УИК №5                                                676                                        

                                             Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования  \
Регион                     ТИК        УИК                                                                                                
Республика Адыгея (Адыгея) Адыгейская УИК №1                                                 62                                          
                                      УИК №2                                                 41                                          
                                      УИК №3                             

Сохраняем как ```elections_results.csv```

In [12]:
df.to_csv("elections_results.csv")